In [2]:
def trade_simulation(initial_balance=100, max_trades=30):
    balance = initial_balance
    trade_results = []
    
    for trade_num in range(1, max_trades + 1):
        risk_percent = (trade_num + 1)  # 1st trade = 2%, 2nd trade = 3%, ...
        risk_amount = (risk_percent / 100) * initial_balance
        
        balance -= risk_amount  # SL hit → deduct
        
        trade_results.append({
            "Trade": trade_num,
            "Risk% of Initial": risk_percent,
            "Risk Amount ($)": round(risk_amount, 2),
            "Balance After Trade ($)": round(balance, 2)
        })
        
        if balance <= 0:
            break  # Stop if account goes negative
    
    return trade_results


In [3]:
import pandas as pd
results = trade_simulation()
df = pd.DataFrame(results)
print(df.to_string(index=False))

 Trade  Risk% of Initial  Risk Amount ($)  Balance After Trade ($)
     1                 2              2.0                     98.0
     2                 3              3.0                     95.0
     3                 4              4.0                     91.0
     4                 5              5.0                     86.0
     5                 6              6.0                     80.0
     6                 7              7.0                     73.0
     7                 8              8.0                     65.0
     8                 9              9.0                     56.0
     9                10             10.0                     46.0
    10                11             11.0                     35.0
    11                12             12.0                     23.0
    12                13             13.0                     10.0
    13                14             14.0                     -4.0


In [5]:
def profitable_k(initial_balance=100, max_k=30):
    results = []
    for K in range(max_k+1):
        losses = sum(i+1 for i in range(1, K+1))
        profit = 2 * (K+2)
        balance = initial_balance - losses + profit
        results.append((K, balance, balance > initial_balance))
    return results

res = profitable_k()
for K, balance, is_profit in res[:100]:  # show first 10
    print(f"K={K} SL then TP -> Balance={balance:.2f}, Profitable={is_profit}")


K=0 SL then TP -> Balance=104.00, Profitable=True
K=1 SL then TP -> Balance=104.00, Profitable=True
K=2 SL then TP -> Balance=103.00, Profitable=True
K=3 SL then TP -> Balance=101.00, Profitable=True
K=4 SL then TP -> Balance=98.00, Profitable=False
K=5 SL then TP -> Balance=94.00, Profitable=False
K=6 SL then TP -> Balance=89.00, Profitable=False
K=7 SL then TP -> Balance=83.00, Profitable=False
K=8 SL then TP -> Balance=76.00, Profitable=False
K=9 SL then TP -> Balance=68.00, Profitable=False
K=10 SL then TP -> Balance=59.00, Profitable=False
K=11 SL then TP -> Balance=49.00, Profitable=False
K=12 SL then TP -> Balance=38.00, Profitable=False
K=13 SL then TP -> Balance=26.00, Profitable=False
K=14 SL then TP -> Balance=13.00, Profitable=False
K=15 SL then TP -> Balance=-1.00, Profitable=False
K=16 SL then TP -> Balance=-16.00, Profitable=False
K=17 SL then TP -> Balance=-32.00, Profitable=False
K=18 SL then TP -> Balance=-49.00, Profitable=False
K=19 SL then TP -> Balance=-67.00, Pro

In [7]:
# Variant A: risk % of INITIAL balance (fixed dollar risk amounts)
def trades_to_target_initial(initial_balance=100.0, target=200.0, block_percents=(2,3,4,5)):
    # dollar risks are based on initial balance (constant)
    risks = [p/100.0 * initial_balance for p in block_percents]
    net_per_block = 2*risks[3] - sum(risks[:3])  # TP on 4th gives 2*risk4
    if net_per_block <= 0:
        return None, None  # never reach target with non-positive block net
    needed_gain = target - initial_balance
    blocks_needed = int((needed_gain + net_per_block - 1e-12) // net_per_block)  # floor
    # ensure we reach or exceed target (round up)
    if blocks_needed * net_per_block < needed_gain:
        blocks_needed += 1
    trades_needed = blocks_needed * len(block_percents)
    final_balance = initial_balance + blocks_needed * net_per_block
    return trades_needed, final_balance

# # Variant B: risk % of CURRENT balance (compounding)
# def trades_to_target_compounding(initial_balance=100.0, target=200.0, block_percents=(2,3,4,5), max_trades=2000000):
#     balance = initial_balance
#     trades = 0
#     while balance < target and trades < max_trades:
#         for i, p in enumerate(block_percents):
#             trades += 1
#             risk = p/100.0 * balance
#             if i < len(block_percents)-1:
#                 # SL on first (n-1) trades
#                 balance -= risk
#             else:
#                 # TP on last (4th) trade: 1:2 RR
#                 balance += 2 * risk
#             if balance >= target:
#                 return trades, balance
#     return None, balance

if __name__ == "__main__":
    t_init, bal_init = trades_to_target_initial()
    # t_comp, bal_comp = trades_to_target_compounding()

    print("Variant A (risk% of INITIAL):")
    if t_init:
        print(f"  Trades needed = {t_init}, final balance = ${bal_init:.2f}")
    else:
        print("  Block net <= 0 → will never reach target.")

    # print("\nVariant B (risk% of CURRENT balance):")
    # if t_comp:
    #     print(f"  Trades needed = {t_comp}, final balance = ${bal_comp:.2f}")
    # else:
    #     print("  Did not reach target within the max iterations.")


Variant A (risk% of INITIAL):
  Trades needed = 400, final balance = $200.00
